In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/My Drive/毕业设计")

In [ ]:
#导入相关的包
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score as CVS
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score as r2
import math

#导入原始数据
y_val = pd.read_csv('train_data_111.csv')

In [ ]:
#数据处理
#将LV1211 S01 PN111_BV25大于65的值作为异常值，并通过插值替代
y_val.loc[y_val['LV1211 S01 PN111_BV25']>=65,'LV1211 S01 PN111_BV25']=np.nan
y_val['LV1211 S01 PN111_BV25']=y_val['LV1211 S01 PN111_BV25'].interpolate()
#同理，将异常值设为nan，并通过插值替代
y_val.loc[y_val['LV1281 AS361 M1']==0,'LV1281 AS361 M1']=np.nan
y_val['LV1281 AS361 M1']=y_val['LV1281 AS361 M1'].interpolate()
y_val.loc[y_val['LV1281 VA307 BB42']==0,'LV1281 VA307 BB42']=np.nan
y_val['LV1281 VA307 BB42']=y_val['LV1281 VA307 BB42'].interpolate()
y_val.loc[y_val['LV1281 VZ366 BB40']==0,'LV1281 VZ366 BB40']=np.nan
y_val['LV1281 VZ366 BB40']=y_val['LV1281 VZ366 BB40'].interpolate()
y_val.loc[y_val['LV1281 WK363 BB40-1']==0,'LV1281 WK363 BB40-1']=np.nan
y_val['LV1281 WK363 BB40-1']=y_val['LV1281 WK363 BB40-1'].interpolate()
y_val.loc[y_val['LV1281 WK363 BB40-2']==0,'LV1281 WK363 BB40-2']=np.nan
y_val['LV1281 WK363 BB40-2']=y_val['LV1281 WK363 BB40-2'].interpolate()

tihuan = ['LV1281 S05 WU301 BB40','LV1281 S05 WU301 BB40-1','LV1281 S06 WU302 BB40','LV1281 S07 WU311 BB40','LV1281 S08 WU312 BB40','LV1281 S09 WU321 BB40']
for item in tihuan:
    y_val.loc[y_val[item] == 0, item] = np.nan
    y_val[item] = y_val[item].interpolate()

tihuan2=['LV1281 S07 WU311 M2','LV1281 S08 WU312 M2','LV1281 S09 WU321 M2','LV1281 S05 WU301 M2','LV1281  S06 WU302 M2']
y_val.loc[73149:73541, tihuan2] = np.nan
for item in tihuan2:
    y_val[item] = y_val[item].interpolate()

y_val.loc[29118:29133,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[37679:37698,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[57740:57755,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[70569:70585,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[78940:78954,['Oven1_NMHC','Oven1_NOx']] = np.nan

y_val.loc[24098:24102,['Oven1_NMHC']] = np.nan
y_val.loc[24118:24122,['Oven1_NMHC']] = np.nan
y_val.loc[74000:74004,['Oven1_NMHC']] = np.nan
y_val['Oven1_NMHC']=y_val['Oven1_NMHC'].interpolate()
y_val['Oven1_NOx']=y_val['Oven1_NOx'].interpolate()

Y_val =y_val.iloc[56886:81076,:].copy()
Y_val1 = y_val.iloc[20268:44076,:].copy()

#线性增长变量做差分
dif_col=['LV1281 S04 WR706 BV25-1','LV1281 S01 Energy consumption','LV1281 S02 Energy consumption','LV1281 S03 Energy consumption']
for item in dif_col:
    ts_diff = np.diff(Y_val[item])
    Y_val[item] = np.append([0], ts_diff)
    ts_diff = np.diff(Y_val1[item])
    Y_val1[item] = np.append([0], ts_diff)

#将两个取出来的时间段Y_val1,Y_val1拼在一块，作为输入的数据集
df1 = pd.concat([Y_val1, Y_val], ignore_index=True)
# bxg=['date','LA1251 summary AmereMin charge','LV1211 S01 PN212_BV25','LV1211 S01 PN213_BV25','LV1281 AS361 M1','LV1281 VA307 M3','LV1281 WK363 BV25','LV1281 WK363 M2']
# df1=df1.drop(columns=bxg)

In [ ]:
#基于相似性
simi_result=['LF1921 S01 TF270 Number of unit', 'LV1281 S04 WA506 BB40-4',
       'LV1281 S04 WA506 BB40-2', 'LV1281 S08 WU312 M2',
       'LV1281 S05 WU301 BB40-1', 'LV1281 S05 WU301 M2',
       'LV1281 S04 WA506 BB40-5', 'LV1281  S06 WU302 M2',
       'LV1281 S07 WU311 M2', 'LF1921 RB260 Forward',
       'LV1211 S01 PN111_BL50', 'LV1281 S04 WR706 M1',
       'LV1281 VA307 BB42', 'LV1281 S06 WU302 M1',
       'LV1211 S01 PN613_BV25', 'LV1281 VA307 M1',
       'LV1281 S04 WA506 BB40-1', 'LV1281 AS361 M1',
       'LF1921 RB260 Occupied', 'LV1211 S01 PR114_BV25','Oven1_NMHC','Oven1_NOx']
df1=df1[simi_result]

In [ ]:
#基于互信息
MI_result=['LV1281 S04 WA506 BB40-5', 'LV1281 VZ366 BB40',
       'LV1281  S06 WU302 M2', 'LV1281 VA307 BB42',
       'LF1921 S01 TF270 Number of unit', 'LV1281 S04 WA506 BB40-2',
       'LV1281 S08 WU312 M2', 'LV1281 S04 WA506 BB40-4',
       'LV1211 S02 PR310_BN20', 'LV1281 S07 WU311 M2',
       'LV1281 S05 WU301 BB40-1', 'LV1211 S03 MW300_BV25-1',
       'LV1211 S01 PN111_BL50', 'LV1281 S09 WU321 M2',
       'LV1281 S05 WU301 M2', 'LV1211 S02 PR313_BB40-1',
       'LV1281 Emergency Cooling Finished', 'LV1211 S01 PN111_BV25',
       'LV1211 S04 VZ503_M1_F', 'LV1211 S01 PR114_BV25','Oven1_NMHC','Oven1_NOx']
df1=df1[MI_result]

In [ ]:
#基于稀疏学习
sparse_result=['LV1281  S06 WU302 M2', 'LF1921 S01 TF270 Home position',
       'LV1211 S01 PK113_BB40', 'LV1281 S06 WU302 M1',
       'LV1281 S04 WA506 BB40-2', 'LV1211 S04 VA515_M1_C',
       'LV1281 S04 WA506 BB40-5', 'LV1211 S04 VZ502_M1_C',
       'LV1281 S08 WU312 M2', 'LF1921 S01 TF270 Forward',
       'LV1281 VA307 BB42', 'LV1281 S05 WU301 M2',
       'LF1921 S01 TF270 Transport forward', 'LV1281 S07 WU311 M2',
       'LV1281 S04 WR706 M1', 'LV1281 VF330 M1',
       'LF1921 RB290 Transport Forward', 'LV1281 WK363 BB40-1',
       'LV1211 S01 PK110_BN20-1', 'LV1211 S01 PN111_BL50','Oven1_NMHC','Oven1_NOx']
df1=df1[MI_result]

In [ ]:
#集成特征选择结果
Final_result=['LV1281 S04 WA506 BB40-2', 'LV1281 S04 WA506 BB40-5',
       'LV1281  S06 WU302 M2', 'LV1281 S08 WU312 M2',
       'LF1921 S01 TF270 Number of unit', 'LV1281 S04 WA506 BB40-4',
       'LV1281 S05 WU301 M2', 'LV1281 VA307 BB42', 'LV1281 S07 WU311 M2',
       'LV1281 S05 WU301 BB40-1', 'LV1211 S01 PN111_BL50',
       'LV1281 S06 WU302 M1', 'LV1281 S04 WR706 M1', 'LV1281 VZ366 BB40',
       'LV1211 S02 PR310_BN20', 'LF1921 RB260 Forward', 'LV1281 VF330 M1',
       'LV1211 S01 PK110_BN20-1', 'LV1211 S01 PK113_BB40',
       'LF1921 S01 TF270 Forward','Oven1_NMHC','Oven1_NOx']
df1=df1[Final_result]      

In [ ]:
#被选择出来的变量
IM=['LV1211 S01 PN111_BL50','LV1211 S04 VZ502_M1_C','LV1281 S04 WR706 M1','LV1281 S04 WA506 BB40-2','LV1281 S04 WA506 BB40-4',
'LV1281 S04 WA506 BB40-5','LV1281 S03 VA306 M1','LV1281 S08 WU312 M2','LV1281 S07 WU311 M2','LV1281 S06 WU302 M1','LV1281  S06 WU302 M2',
'LV1281 S05 WU301 M2','LV1281 S05 WU301 BB40-1','LV1281 VZ366 BB40','LV1281 VA307 BB42','LF1921 S01 TF270 Number of unit','LF1921 S01 TF270 Transport forward',
'LF1921 S01 TF270 Home position','LF1921 RB280 Forward','LF1921 RB290 Occupied','Oven1_NMHC','Oven1_NOx']
df1=df1[IM]

In [ ]:
#用于深度强化学习的18个关键特征+烘房车辆数目
plot_variable=['LV1281 S04 WA506 BB40-5', 'LV1281 VZ366 BB40',
       'LV1281  S06 WU302 M2', 'LV1281 VA307 BB42',
        'LV1281 S04 WA506 BB40-2',
       'LV1281 S08 WU312 M2', 'LV1281 S04 WA506 BB40-4',
       'LV1211 S02 PR310_BN20', 'LV1281 S07 WU311 M2',
       'LV1281 S05 WU301 BB40-1','LV1211 S01 PN111_BL50',
       'LV1281 S09 WU321 M2',
       'LV1281 S05 WU301 M2', 'LV1211 S02 PR313_BB40-1',
        'LV1211 S01 PN111_BV25',
       'LV1211 S04 VZ503_M1_F', 'LV1211 S01 PR114_BV25',
       'LF1921 S01 TF270 Number of unit','Oven1_NMHC','Oven1_NOx']
df1=df1[plot_variable]

In [ ]:
#取前30分钟和前90到60的数据作为输入
def feature_engineer(df,name):
    N=30
    look_back=25
    skip = 60
    N2=30
    M=20
    lag_cols = [column for column in df][0:]#得到相应的列名
    shift_range = [x + 1 for x in range(N)]#生成1到30的数
    
    for i in shift_range:
        for col in lag_cols:
            new_col='{}_lag_{}'.format(col, i)   # 格式化字符串
            df[new_col]=df[col].shift(i)
    # shift_range2 = [x + 61 for x in range(N2)]
    # for col in lag_cols:
    #     for i in shift_range2:
    #         new_col='{}_lag_{}'.format(col, i)   # 格式化字符串
    #         df[new_col]=df[col].shift(i)
    # for i in shift_range:
    #     new_col = '{}_lag_{}'.format(col, i)  # 格式化字符串
    #     df[new_col] = df[col].shift(i)
    shift_range_y = [x + 1 for x in range(M)]#生成1到20的数
    for i in shift_range_y: 
        new_col = '{}_pre_{}'.format(name, -i)#格式化字符串
        df[new_col] = df[name].shift(-i)
        
    return df[look_back*skip:-M-1]
# df = feature_engineer(df1,'Oven1_NOx')
df = feature_engineer(df1,'Oven1_NOx')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
continuous = [i for i in df.loc[:,df.nunique()>2]][0:]
categorical = [i for i in df.loc[:,df.nunique()<=2]]

In [ ]:
#分割训练集与测试集
test_size = 0.3
num_test = int(test_size * len(df))
num_train = len(df)-num_test
train = df[:num_train]
test = df[num_train:]

# 数据标准化：标准化也不应带测试集，以避免信息泄漏
scaler = StandardScaler().fit(train[continuous])
train_scaled = scaler.transform(train[continuous])
test_scaled = scaler.transform(test[continuous])

# Convert the numpy array back into pandas dataframe
train_scaled = pd.DataFrame(train_scaled, columns=continuous)
test_scaled = pd.DataFrame(test_scaled, columns=continuous)

#得到处理后的训练集和测试集数据
train.reset_index(drop=True, inplace=True)
train_scaled.reset_index(drop=True, inplace=True)
temp_train = pd.concat([train[categorical],train_scaled],axis=1,join='inner')

test.reset_index(drop=True, inplace=True)
test_scaled.reset_index(drop=True, inplace=True)
temp_test = pd.concat([test[categorical],test_scaled],axis=1,join='inner')

In [ ]:
X_train_scaled = temp_train.iloc[:,0:-20]
Y_train_scaled = temp_train.iloc[:,-20:]
X_test_scaled = temp_test.iloc[:,0:-20]
Y_test_scaled = temp_test.iloc[:,-20:]

In [ ]:
#LSTM模型
# reshape input to be [samples, time steps, features]
x_train = X_train_scaled.values
x_test = X_test_scaled.values
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
y_train = Y_train_scaled.values
y_test = Y_test_scaled.values

from keras.models import Sequential
from keras.layers import Dense, LSTM
import keras
import tensorflow as tf
from tensorflow import keras as K

In [ ]:
y_train[0]

-1.2895219637303925

In [ ]:
#写一个LossHistory类，保存loss和acc
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # # acc
        # plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'dodgerblue', label='train loss')
        if loss_type == 'epoch':
            # # val_acc
            # plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'darkorange', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('loss')
        plt.legend(loc="upper right")
        plt.savefig('loss{0}'.format(loss_type))

    def acc_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'dodgerblue', label='train acc')
        # loss
        # plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'darkorange', label='val acc')
            # val_loss
            # plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc')
        plt.legend(loc="upper right")
        plt.savefig('acc{0}'.format(loss_type))

In [ ]:
x_train.shape[1], x_train.shape[2]

(1, 2790)

In [ ]:
start = datetime.datetime.now()
#Build the LSTM model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(64))
model.add(Dense(20))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

#创建室类
history = LossHistory()
#Train the model

# model.fit(x_train, y_train, batch_size=200, epochs=70, validation_data=(x_test, y_test),callbacks=[history])
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test),callbacks=[history])


model.summary()

# history.loss_plot('epoch')
# history.acc_plot('epoch')
# Lets predict with the model
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)
score = model.evaluate(x_test, y_test)

end = datetime.datetime.now()
print("程序运行时间："+str((end-start).seconds)+"秒")

In [ ]:
from keras.models import load_model

model.save('lstm_model_NOx.h5')

In [ ]:
array=np.array([i for i in range(465)])
train_scaled = scaler.transform(array.reshape(1, -1))

In [ ]:
train_scaled=scaler.transform(train_scaled)

In [ ]:
scaler = StandardScaler().fit(train[continuous].iloc[:,0:-20])
array=np.array([i for i in range(465)])
train_scaled = scaler.transform(array.reshape(1, -1))
train_scaled = np.reshape(train_scaled, (train_scaled.shape[0], 1, train_scaled.shape[1]))
train_predict = model.predict(train_scaled)

In [ ]:
array=np.array([i for i in range(465)])
train_scaled = scaler.transform(array.reshape(1, -1))

In [ ]:
model = load_model('lstm_model_NMHC.h5')
test_predict = model.predict(x_test)

In [ ]:
pre=[]
pre.append(train['Oven1_NOx_pre_-20'].min())
pre.append(train['Oven1_NOx_pre_-20'].max())

In [ ]:
pre=[]
pre.append(train['Oven1_NMHC_pre_-20'].min())
pre.append(train['Oven1_NMHC_pre_-20'].max())

极大值极小值标准化

In [ ]:
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# Get the root mean squared error (RMSE) and MAE
# print(test_predict[:,0]*pre[1]+pre[0])
# print(test['Oven1_NOx_pre_-20'])

score_rmse = np.sqrt(MSE(test['Oven1_NOx_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0])))
score_mae = MAE(test['Oven1_NOx_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_mae1 = MAE(train['Oven1_NOx_pre_-20'].values, pre[0]+train_predict[:,0]*(pre[1]-pre[0]))
score_me=ME(test['Oven1_NOx_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_r2=r2(test['Oven1_NOx_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_smape = smape(test['Oven1_NOx_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))

print('score_rmse:',score_rmse)
print('score_mae:',score_mae)
print('score_me:',score_me)
print('score_r2:',score_r2)
print('score_smape:',score_smape)
a = np.argmax(test['Oven1_NOx_pre_-20'].values)
print(np.max(test['Oven1_NOx_pre_-20'].values)-np.max(test_predict[a-20:a+5,0]*pre[1]+pre[0]))
print('最大值差：',np.max(test['Oven1_NOx_pre_-20'].values)-np.max(test_predict[:,0]*pre[1]+pre[0]))

In [ ]:
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

# Get the root mean squared error (RMSE) and MAE
# print(test_predict[:,0]*pre[1]+pre[0])
# print(test['Oven1_NOx_pre_-20'])

score_rmse = np.sqrt(MSE(test['Oven1_NMHC_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0])))
score_mae = MAE(test['Oven1_NMHC_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_mae1 = MAE(train['Oven1_NMHC_pre_-20'].values, pre[0]+train_predict[:,0]*(pre[1]-pre[0]))
score_me=ME(test['Oven1_NMHC_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_r2=r2(test['Oven1_NMHC_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))
score_smape = smape(test['Oven1_NMHC_pre_-20'].values, pre[0]+test_predict[:,0]*(pre[1]-pre[0]))

print('score_rmse:',score_rmse)
print('score_mae:',score_mae)
print('score_me:',score_me)
print('score_r2:',score_r2)
print('score_smape:',score_smape)
a = np.argmax(test['Oven1_NMHC_pre_-20'].values)
print(np.max(test['Oven1_NMHC_pre_-20'].values)-np.max(test_predict[a-20:a+5,0]*pre[1]+pre[0]))
print('最大值差：',np.max(test['Oven1_NMHC_pre_-20'].values)-np.max(test_predict[:,0]*pre[1]+pre[0]))

均值标准差标准化

In [ ]:
#NOx
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100


pre=[]
pre.append(train['Oven1_NMHC_pre_-20'].mean())
pre.append(train['Oven1_NMHC_pre_-20'].std())
rmse_=[]
mae_=[]
r2_=[]
smape_=[]

for i in range(1,21):
  score_rmse = np.sqrt(MSE(test['Oven1_NMHC_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0]))
  score_mae = MAE(test['Oven1_NMHC_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  score_r2=r2(test['Oven1_NMHC_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  score_smape = smape(test['Oven1_NMHC_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  rmse_.append(score_rmse)
  mae_.append(score_mae)
  r2_.append(score_r2)
  smape_.append(score_smape)

print('score_rmse:',np.mean(rmse_))
print('score_mae:',np.mean(mae_))
print('score_r2:',np.mean(r2_))
print('score_smape:',np.mean(smape_))
print("_______________________________________________________")
print('score_rmse:',rmse_[0])
print('score_mae:',mae_[0])
print('score_r2:',r2_[0])
print('score_smape:',smape_[0])

score_rmse: 0.14511945275963606
score_mae: 0.0831311682484024
score_r2: 0.919615499011883
score_smape: 33.95846388900826
_______________________________________________________
score_rmse: 0.0971641862873524
score_mae: 0.06546790146242895
score_r2: 0.9643767349711637
score_smape: 32.15561803345637


In [ ]:
#NOx
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100


pre=[]
pre.append(train['Oven1_NOx_pre_-20'].mean())
pre.append(train['Oven1_NOx_pre_-20'].std())
rmse_=[]
mae_=[]
r2_=[]
smape_=[]

for i in range(1,21):
  score_rmse = np.sqrt(MSE(test['Oven1_NOx_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0]))
  score_mae = MAE(test['Oven1_NOx_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  score_r2=r2(test['Oven1_NOx_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  score_smape = smape(test['Oven1_NOx_pre_-%s'%i].values, test_predict[:,i-1]*pre[1]+pre[0])
  rmse_.append(score_rmse)
  mae_.append(score_mae)
  r2_.append(score_r2)
  smape_.append(score_smape)

print('score_rmse:',np.mean(rmse_))
print('score_mae:',np.mean(mae_))
print('score_r2:',np.mean(r2_))
print('score_smape:',np.mean(smape_))
print("_______________________________________________________")
print('score_rmse:',rmse_[0])
print('score_mae:',mae_[0])
print('score_r2:',r2_[0])
print('score_smape:',smape_[0])

score_rmse: 2.6135137184879573
score_mae: 1.1688441832796304
score_r2: 0.9866098899327482
score_smape: 8.565537142947212
_______________________________________________________
score_rmse: 1.5336716492788964
score_mae: 0.898271568898054
score_r2: 0.9954799705460501
score_smape: 6.264900224791015
